In [1]:
#读取csv格式文件
import pandas as pd
path = './' #用户的路径环境
train = pd.read_csv(path+'train_set.csv')
test = pd.read_csv(path+'test_set.csv')
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [2]:
train.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome,y
0,1,43,management,married,tertiary,no,291,yes,no,unknown,9,may,150,2,-1,0,unknown,0
1,2,42,technician,divorced,primary,no,5076,yes,no,cellular,7,apr,99,1,251,2,other,0
2,3,47,admin.,married,secondary,no,104,yes,yes,cellular,14,jul,77,2,-1,0,unknown,0
3,4,28,management,single,secondary,no,-994,yes,yes,cellular,18,jul,174,2,-1,0,unknown,0
4,5,42,technician,divorced,secondary,no,2974,yes,no,unknown,21,may,187,5,-1,0,unknown,0


In [3]:
test.head()

,ID,age,job,marital,education,default,balance,housing,loan,contact,day,month,duration,campaign,pdays,previous,poutcome
0,25318,51,housemaid,married,unknown,no,174,no,no,telephone,29,jul,308,3,-1,0,unknown
1,25319,32,management,married,tertiary,no,6059,yes,no,cellular,20,nov,110,2,-1,0,unknown
2,25320,60,retired,married,primary,no,0,no,no,telephone,30,jul,130,3,-1,0,unknown
3,25321,32,student,single,tertiary,no,64,no,no,cellular,30,jun,598,4,105,5,failure
4,25322,41,housemaid,married,secondary,no,0,yes,yes,cellular,15,jul,368,4,-1,0,unknown


In [4]:
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
from scipy import sparse
import lightgbm as lgb
test['y']=-1

cate_features = ['job', 'marital','education','default','housing','loan','contact','month','poutcome']
num_features = ['age', 'balance','day','duration','campaign','pdays','previous']


feature1 = cate_features + num_features
data = pd.concat([train,test])

def feat_count(df, df_feature,fe,value,name=""):
    df_count = pd.DataFrame(df_feature.groupby(fe)[value].count()).reset_index()
    if not name:
        df_count.columns = fe + [value+"_%s_count" % ("_".join(fe[0]))]
    else:
        df_count.columns = fe + [name]
    df = df.merge(df_count, on=fe, how="left")#.fillna(0)
    return df,df_count.columns[1]
for df in feature1:
    data = feat_count(data,data,['ID'],df)[0]

data.head()

def feature_count(data, features):
    feature_name = 'count'
    for i in features:
        feature_name += '_' + i
    temp = data.groupby(features).size().reset_index().rename(columns={0: feature_name})
    data = data.merge(temp, 'left', on=features)
    return data,feature_name


ll=[]
for f in['campaign', 'contact','default','education','housing','job','loan','marital','poutcome','pdays','previous']:
    data,_=feature_count(data,['day','month',f])
    ll.append(_)
sparse_feature= ['campaign','contact','default','education','housing','job','loan','marital','month','poutcome']
dense_feature=['age', 'balance','day','duration','campaign','pdays','previous']

def get_new_columns(name,aggs):
    l=[]
    for k in aggs.keys():
        for agg in aggs[k]:
            if str(type(agg))=="<class 'function'>":
                l.append(name + '_' + k + '_' + 'other')
            else:
                l.append(name + '_' + k + '_' + agg)
    return l
for d in tqdm(sparse_feature):
    aggs={}
    for s in sparse_feature:
        aggs[s]=['count','nunique']
    for den in dense_feature:
        aggs[den]=['mean','max','min','std']
    aggs.pop(d)
    temp=data.groupby(d).agg(aggs).reset_index()
    temp.columns=[d]+get_new_columns(d,aggs)
    data=pd.merge(data,temp,on=d,how='left')


100%|██████████████████████████████████████████████████████████████████████████████████| 10/10 [00:24<00:00,  4.01s/it]


In [5]:
df_train_columns=data.columns.tolist()

In [6]:
df_train_columns

['ID',
 'age',
 'job',
 'marital',
 'education',
 'default',
 'balance',
 'housing',
 'loan',
 'contact',
 'day',
 'month',
 'duration',
 'campaign',
 'pdays',
 'previous',
 'poutcome',
 'y',
 'job_I_D_count',
 'marital_I_D_count',
 'education_I_D_count',
 'default_I_D_count',
 'housing_I_D_count',
 'loan_I_D_count',
 'contact_I_D_count',
 'month_I_D_count',
 'poutcome_I_D_count',
 'age_I_D_count',
 'balance_I_D_count',
 'day_I_D_count',
 'duration_I_D_count',
 'campaign_I_D_count',
 'pdays_I_D_count',
 'previous_I_D_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nun

In [7]:
#ID这里好像没用'ID',
import numpy as np

from scipy.sparse import csr_matrix

from sklearn.model_selection import KFold

num_features=['job_I_D_count',
 'marital_I_D_count',
 'education_I_D_count',
 'default_I_D_count',
 'housing_I_D_count',
 'loan_I_D_count',
 'contact_I_D_count',
 'month_I_D_count',
 'poutcome_I_D_count',
 'age_I_D_count',
 'balance_I_D_count',
 'day_I_D_count',
 'duration_I_D_count',
 'campaign_I_D_count',
 'pdays_I_D_count',
 'previous_I_D_count',
 'count_day_month_campaign',
 'count_day_month_contact',
 'count_day_month_default',
 'count_day_month_education',
 'count_day_month_housing',
 'count_day_month_job',
 'count_day_month_loan',
 'count_day_month_marital',
 'count_day_month_poutcome',
 'count_day_month_pdays',
 'count_day_month_previous',
 'campaign_contact_count',
 'campaign_contact_nunique',
 'campaign_default_count',
 'campaign_default_nunique',
 'campaign_education_count',
 'campaign_education_nunique',
 'campaign_housing_count',
 'campaign_housing_nunique',
 'campaign_job_count',
 'campaign_job_nunique',
 'campaign_loan_count',
 'campaign_loan_nunique',
 'campaign_marital_count',
 'campaign_marital_nunique',
 'campaign_month_count',
 'campaign_month_nunique',
 'campaign_poutcome_count',
 'campaign_poutcome_nunique',
 'campaign_age_mean',
 'campaign_age_max',
 'campaign_age_min',
 'campaign_age_std',
 'campaign_balance_mean',
 'campaign_balance_max',
 'campaign_balance_min',
 'campaign_balance_std',
 'campaign_day_mean',
 'campaign_day_max',
 'campaign_day_min',
 'campaign_day_std',
 'campaign_duration_mean',
 'campaign_duration_max',
 'campaign_duration_min',
 'campaign_duration_std',
 'campaign_pdays_mean',
 'campaign_pdays_max',
 'campaign_pdays_min',
 'campaign_pdays_std',
 'campaign_previous_mean',
 'campaign_previous_max',
 'campaign_previous_min',
 'campaign_previous_std',
 'contact_campaign_mean',
 'contact_campaign_max',
 'contact_campaign_min',
 'contact_campaign_std',
 'contact_default_count',
 'contact_default_nunique',
 'contact_education_count',
 'contact_education_nunique',
 'contact_housing_count',
 'contact_housing_nunique',
 'contact_job_count',
 'contact_job_nunique',
 'contact_loan_count',
 'contact_loan_nunique',
 'contact_marital_count',
 'contact_marital_nunique',
 'contact_month_count',
 'contact_month_nunique',
 'contact_poutcome_count',
 'contact_poutcome_nunique',
 'contact_age_mean',
 'contact_age_max',
 'contact_age_min',
 'contact_age_std',
 'contact_balance_mean',
 'contact_balance_max',
 'contact_balance_min',
 'contact_balance_std',
 'contact_day_mean',
 'contact_day_max',
 'contact_day_min',
 'contact_day_std',
 'contact_duration_mean',
 'contact_duration_max',
 'contact_duration_min',
 'contact_duration_std',
 'contact_pdays_mean',
 'contact_pdays_max',
 'contact_pdays_min',
 'contact_pdays_std',
 'contact_previous_mean',
 'contact_previous_max',
 'contact_previous_min',
 'contact_previous_std',
 'default_campaign_mean',
 'default_campaign_max',
 'default_campaign_min',
 'default_campaign_std',
 'default_contact_count',
 'default_contact_nunique',
 'default_education_count',
 'default_education_nunique',
 'default_housing_count',
 'default_housing_nunique',
 'default_job_count',
 'default_job_nunique',
 'default_loan_count',
 'default_loan_nunique',
 'default_marital_count',
 'default_marital_nunique',
 'default_month_count',
 'default_month_nunique',
 'default_poutcome_count',
 'default_poutcome_nunique',
 'default_age_mean',
 'default_age_max',
 'default_age_min',
 'default_age_std',
 'default_balance_mean',
 'default_balance_max',
 'default_balance_min',
 'default_balance_std',
 'default_day_mean',
 'default_day_max',
 'default_day_min',
 'default_day_std',
 'default_duration_mean',
 'default_duration_max',
 'default_duration_min',
 'default_duration_std',
 'default_pdays_mean',
 'default_pdays_max',
 'default_pdays_min',
 'default_pdays_std',
 'default_previous_mean',
 'default_previous_max',
 'default_previous_min',
 'default_previous_std',
 'education_campaign_mean',
 'education_campaign_max',
 'education_campaign_min',
 'education_campaign_std',
 'education_contact_count',
 'education_contact_nunique',
 'education_default_count',
 'education_default_nunique',
 'education_housing_count',
 'education_housing_nunique',
 'education_job_count',
 'education_job_nunique',
 'education_loan_count',
 'education_loan_nunique',
 'education_marital_count',
 'education_marital_nunique',
 'education_month_count',
 'education_month_nunique',
 'education_poutcome_count',
 'education_poutcome_nunique',
 'education_age_mean',
 'education_age_max',
 'education_age_min',
 'education_age_std',
 'education_balance_mean',
 'education_balance_max',
 'education_balance_min',
 'education_balance_std',
 'education_day_mean',
 'education_day_max',
 'education_day_min',
 'education_day_std',
 'education_duration_mean',
 'education_duration_max',
 'education_duration_min',
 'education_duration_std',
 'education_pdays_mean',
 'education_pdays_max',
 'education_pdays_min',
 'education_pdays_std',
 'education_previous_mean',
 'education_previous_max',
 'education_previous_min',
 'education_previous_std',
 'housing_campaign_mean',
 'housing_campaign_max',
 'housing_campaign_min',
 'housing_campaign_std',
 'housing_contact_count',
 'housing_contact_nunique',
 'housing_default_count',
 'housing_default_nunique',
 'housing_education_count',
 'housing_education_nunique',
 'housing_job_count',
 'housing_job_nunique',
 'housing_loan_count',
 'housing_loan_nunique',
 'housing_marital_count',
 'housing_marital_nunique',
 'housing_month_count',
 'housing_month_nunique',
 'housing_poutcome_count',
 'housing_poutcome_nunique',
 'housing_age_mean',
 'housing_age_max',
 'housing_age_min',
 'housing_age_std',
 'housing_balance_mean',
 'housing_balance_max',
 'housing_balance_min',
 'housing_balance_std',
 'housing_day_mean',
 'housing_day_max',
 'housing_day_min',
 'housing_day_std',
 'housing_duration_mean',
 'housing_duration_max',
 'housing_duration_min',
 'housing_duration_std',
 'housing_pdays_mean',
 'housing_pdays_max',
 'housing_pdays_min',
 'housing_pdays_std',
 'housing_previous_mean',
 'housing_previous_max',
 'housing_previous_min',
 'housing_previous_std',
 'job_campaign_mean',
 'job_campaign_max',
 'job_campaign_min',
 'job_campaign_std',
 'job_contact_count',
 'job_contact_nunique',
 'job_default_count',
 'job_default_nunique',
 'job_education_count',
 'job_education_nunique',
 'job_housing_count',
 'job_housing_nunique',
 'job_loan_count',
 'job_loan_nunique',
 'job_marital_count',
 'job_marital_nunique',
 'job_month_count',
 'job_month_nunique',
 'job_poutcome_count',
 'job_poutcome_nunique',
 'job_age_mean',
 'job_age_max',
 'job_age_min',
 'job_age_std',
 'job_balance_mean',
 'job_balance_max',
 'job_balance_min',
 'job_balance_std',
 'job_day_mean',
 'job_day_max',
 'job_day_min',
 'job_day_std',
 'job_duration_mean',
 'job_duration_max',
 'job_duration_min',
 'job_duration_std',
 'job_pdays_mean',
 'job_pdays_max',
 'job_pdays_min',
 'job_pdays_std',
 'job_previous_mean',
 'job_previous_max',
 'job_previous_min',
 'job_previous_std',
 'loan_campaign_mean',
 'loan_campaign_max',
 'loan_campaign_min',
 'loan_campaign_std',
 'loan_contact_count',
 'loan_contact_nunique',
 'loan_default_count',
 'loan_default_nunique',
 'loan_education_count',
 'loan_education_nunique',
 'loan_housing_count',
 'loan_housing_nunique',
 'loan_job_count',
 'loan_job_nunique',
 'loan_marital_count',
 'loan_marital_nunique',
 'loan_month_count',
 'loan_month_nunique',
 'loan_poutcome_count',
 'loan_poutcome_nunique',
 'loan_age_mean',
 'loan_age_max',
 'loan_age_min',
 'loan_age_std',
 'loan_balance_mean',
 'loan_balance_max',
 'loan_balance_min',
 'loan_balance_std',
 'loan_day_mean',
 'loan_day_max',
 'loan_day_min',
 'loan_day_std',
 'loan_duration_mean',
 'loan_duration_max',
 'loan_duration_min',
 'loan_duration_std',
 'loan_pdays_mean',
 'loan_pdays_max',
 'loan_pdays_min',
 'loan_pdays_std',
 'loan_previous_mean',
 'loan_previous_max',
 'loan_previous_min',
 'loan_previous_std',
 'marital_campaign_mean',
 'marital_campaign_max',
 'marital_campaign_min',
 'marital_campaign_std',
 'marital_contact_count',
 'marital_contact_nunique',
 'marital_default_count',
 'marital_default_nunique',
 'marital_education_count',
 'marital_education_nunique',
 'marital_housing_count',
 'marital_housing_nunique',
 'marital_job_count',
 'marital_job_nunique',
 'marital_loan_count',
 'marital_loan_nunique',
 'marital_month_count',
 'marital_month_nunique',
 'marital_poutcome_count',
 'marital_poutcome_nunique',
 'marital_age_mean',
 'marital_age_max',
 'marital_age_min',
 'marital_age_std',
 'marital_balance_mean',
 'marital_balance_max',
 'marital_balance_min',
 'marital_balance_std',
 'marital_day_mean',
 'marital_day_max',
 'marital_day_min',
 'marital_day_std',
 'marital_duration_mean',
 'marital_duration_max',
 'marital_duration_min',
 'marital_duration_std',
 'marital_pdays_mean',
 'marital_pdays_max',
 'marital_pdays_min',
 'marital_pdays_std',
 'marital_previous_mean',
 'marital_previous_max',
 'marital_previous_min',
 'marital_previous_std',
 'month_campaign_mean',
 'month_campaign_max',
 'month_campaign_min',
 'month_campaign_std',
 'month_contact_count',
 'month_contact_nunique',
 'month_default_count',
 'month_default_nunique',
 'month_education_count',
 'month_education_nunique',
 'month_housing_count',
 'month_housing_nunique',
 'month_job_count',
 'month_job_nunique',
 'month_loan_count',
 'month_loan_nunique',
 'month_marital_count',
 'month_marital_nunique',
 'month_poutcome_count',
 'month_poutcome_nunique',
 'month_age_mean',
 'month_age_max',
 'month_age_min',
 'month_age_std',
 'month_balance_mean',
 'month_balance_max',
 'month_balance_min',
 'month_balance_std',
 'month_day_mean',
 'month_day_max',
 'month_day_min',
 'month_day_std',
 'month_duration_mean',
 'month_duration_max',
 'month_duration_min',
 'month_duration_std',
 'month_pdays_mean',
 'month_pdays_max',
 'month_pdays_min',
 'month_pdays_std',
 'month_previous_mean',
 'month_previous_max',
 'month_previous_min',
 'month_previous_std',
 'poutcome_campaign_mean',
 'poutcome_campaign_max',
 'poutcome_campaign_min',
 'poutcome_campaign_std',
 'poutcome_contact_count',
 'poutcome_contact_nunique',
 'poutcome_default_count',
 'poutcome_default_nunique',
 'poutcome_education_count',
 'poutcome_education_nunique',
 'poutcome_housing_count',
 'poutcome_housing_nunique',
 'poutcome_job_count',
 'poutcome_job_nunique',
 'poutcome_loan_count',
 'poutcome_loan_nunique',
 'poutcome_marital_count',
 'poutcome_marital_nunique',
 'poutcome_month_count',
 'poutcome_month_nunique',
 'poutcome_age_mean',
 'poutcome_age_max',
 'poutcome_age_min',
 'poutcome_age_std',
 'poutcome_balance_mean',
 'poutcome_balance_max',
 'poutcome_balance_min',
 'poutcome_balance_std',
 'poutcome_day_mean',
 'poutcome_day_max',
 'poutcome_day_min',
 'poutcome_day_std',
 'poutcome_duration_mean',
 'poutcome_duration_max',
 'poutcome_duration_min',
 'poutcome_duration_std',
 'poutcome_pdays_mean',
 'poutcome_pdays_max',
 'poutcome_pdays_min',
 'poutcome_pdays_std',
 'poutcome_previous_mean',
 'poutcome_previous_max',
 'poutcome_previous_min',
 'poutcome_previous_std']+num_features


one_hot_feature =['job', 'marital','education','default','housing','loan','contact','month','poutcome']


# def encode_count(df,column_name):
#     lbl = preprocessing.LabelEncoder()
#     lbl.fit(list(df[column_name].values))
#     df[column_name] = lbl.transform(list(df[column_name].values))
#     return df

data['new_con'] = data['job'].astype(str)
for i in ['marital', 'education', 'contact','month','poutcome']:
    data['new_con'] = data['new_con'].astype(str) + '_' + data[i].astype(str)
data['new_con'] = data['new_con'].apply(lambda x: ' '.join(x.split('_')))





for feature in one_hot_feature:
    try:
        data[feature] = LabelEncoder().fit_transform(data[feature].apply(int))
    except:
        data[feature] = LabelEncoder().fit_transform(data[feature])
        
vector_feature = ['new_con']


train=data[data.y!=-1]

# train, test, train_y, test_y = train_test_split(train,train_y,test_size=0.2, random_state=2018)
test=data[data.y==-1]
res=test[['ID']]
# test=test.drop(['ID'],axis=1)
train_x=train[num_features]
test_x=test[num_features]
enc = OneHotEncoder()
for feature in one_hot_feature:
    enc.fit(data[feature].values.reshape(-1, 1))
    train_a=enc.transform(train[feature].values.reshape(-1, 1))
    test_a = enc.transform(test[feature].values.reshape(-1, 1))
    train_x= sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')


cv=CountVectorizer()
for feature in vector_feature:
    cv.fit(data[feature])
    train_a = cv.transform(train[feature])
    test_a = cv.transform(test[feature])
    train_x = sparse.hstack((train_x, train_a), 'csr')
    test_x = sparse.hstack((test_x, test_a), 'csr')

# trainxx, vali_x, trainyy, vali_y = train_test_split(train_x,train_y,test_size=0.2, random_state=2018)
n_splits=5
kfold = KFold(n_splits=5, shuffle=True, random_state=42)
clf = lgb.LGBMClassifier(
        boosting_type="gbdt", num_leaves=30, reg_alpha=0, reg_lambda=0.,
    max_depth=-1, n_estimators=1500, objective='binary',metric= 'auc',
    subsample=0.95, colsample_bytree=0.7, subsample_freq=1,
    learning_rate=0.02, random_state=2017
    )

train_y=train['y']
res['pred'] = 0
for train_idx, val_idx in kfold.split(train_x):
    clf.random_state = clf.random_state + 1
    train_x1 = train_x[train_idx]
    train_y1 = train_y.loc[train_idx]
    test_x1 = train_x[val_idx]
    test_y1 = train_y.loc[val_idx]
    #,(vali_x,vali_y)
    clf.fit(train_x1, train_y1,eval_set=[(train_x1, train_y1),(test_x1, test_y1)],eval_metric='auc',early_stopping_rounds=100)
    res['pred'] += clf.predict_proba(test_x)[:,1]

res['pred'] = res['pred']/5
res.to_csv('./basesubmission.csv', index=False)






d:\anaconda3\lib\site-packages\ipykernel_launcher.py:538: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[1]	valid_0's auc: 0.778652	valid_1's auc: 0.800393
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.924345	valid_1's auc: 0.915024
[3]	valid_0's auc: 0.930799	valid_1's auc: 0.920141
[4]	valid_0's auc: 0.929131	valid_1's auc: 0.917078
[5]	valid_0's auc: 0.933418	valid_1's auc: 0.922879
[6]	valid_0's auc: 0.937966	valid_1's auc: 0.928246
[7]	valid_0's auc: 0.936544	valid_1's auc: 0.927411
[8]	valid_0's auc: 0.935953	valid_1's auc: 0.929128
[9]	valid_0's auc: 0.934228	valid_1's auc: 0.927784
[10]	valid_0's auc: 0.932613	valid_1's auc: 0.925983
[11]	valid_0's auc: 0.930665	valid_1's auc: 0.923669
[12]	valid_0's auc: 0.934015	valid_1's auc: 0.926851
[13]	valid_0's auc: 0.936086	valid_1's auc: 0.928392
[14]	valid_0's auc: 0.9381	valid_1's auc: 0.930488
[15]	valid_0's auc: 0.940404	valid_1's auc: 0.932199
[16]	valid_0's auc: 0.941169	valid_1's auc: 0.932965
[17]	valid_0's auc: 0.941928	valid_1's auc: 0.933192
[18]	valid_0's auc: 0.942445	valid_1's auc: 0.9

[306]	valid_0's auc: 0.972741	valid_1's auc: 0.939793
[307]	valid_0's auc: 0.972793	valid_1's auc: 0.939775
[308]	valid_0's auc: 0.972858	valid_1's auc: 0.939809
[309]	valid_0's auc: 0.972927	valid_1's auc: 0.939825
[310]	valid_0's auc: 0.972991	valid_1's auc: 0.939797
[311]	valid_0's auc: 0.973048	valid_1's auc: 0.939774
[312]	valid_0's auc: 0.973132	valid_1's auc: 0.93975
[313]	valid_0's auc: 0.973183	valid_1's auc: 0.939743
[314]	valid_0's auc: 0.973263	valid_1's auc: 0.939792
[315]	valid_0's auc: 0.973341	valid_1's auc: 0.939763
[316]	valid_0's auc: 0.973396	valid_1's auc: 0.939783
[317]	valid_0's auc: 0.973476	valid_1's auc: 0.939798
[318]	valid_0's auc: 0.973551	valid_1's auc: 0.939775
[319]	valid_0's auc: 0.973598	valid_1's auc: 0.939788
[320]	valid_0's auc: 0.973669	valid_1's auc: 0.939808
[321]	valid_0's auc: 0.973732	valid_1's auc: 0.939816
[322]	valid_0's auc: 0.973834	valid_1's auc: 0.939812
[323]	valid_0's auc: 0.973907	valid_1's auc: 0.939828
[324]	valid_0's auc: 0.973997

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:547: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[1]	valid_0's auc: 0.898692	valid_1's auc: 0.889224
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.914952	valid_1's auc: 0.905031
[3]	valid_0's auc: 0.93307	valid_1's auc: 0.918324
[4]	valid_0's auc: 0.931876	valid_1's auc: 0.915547
[5]	valid_0's auc: 0.927768	valid_1's auc: 0.910715
[6]	valid_0's auc: 0.934617	valid_1's auc: 0.916113
[7]	valid_0's auc: 0.938744	valid_1's auc: 0.919513
[8]	valid_0's auc: 0.939835	valid_1's auc: 0.920604
[9]	valid_0's auc: 0.940367	valid_1's auc: 0.921496
[10]	valid_0's auc: 0.940842	valid_1's auc: 0.921556
[11]	valid_0's auc: 0.941942	valid_1's auc: 0.922225
[12]	valid_0's auc: 0.941678	valid_1's auc: 0.921176
[13]	valid_0's auc: 0.942556	valid_1's auc: 0.922586
[14]	valid_0's auc: 0.943045	valid_1's auc: 0.923001
[15]	valid_0's auc: 0.943858	valid_1's auc: 0.923847
[16]	valid_0's auc: 0.943651	valid_1's auc: 0.923789
[17]	valid_0's auc: 0.943594	valid_1's auc: 0.923415
[18]	valid_0's auc: 0.943988	valid_1's auc: 0.

...

[306]	valid_0's auc: 0.972777	valid_1's auc: 0.943612
[307]	valid_0's auc: 0.97282	valid_1's auc: 0.943625
[308]	valid_0's auc: 0.972897	valid_1's auc: 0.94364
[309]	valid_0's auc: 0.97298	valid_1's auc: 0.943669
[310]	valid_0's auc: 0.973072	valid_1's auc: 0.943662
[311]	valid_0's auc: 0.97312	valid_1's auc: 0.943661
[312]	valid_0's auc: 0.973179	valid_1's auc: 0.943656
[313]	valid_0's auc: 0.973259	valid_1's auc: 0.943662
[314]	valid_0's auc: 0.973322	valid_1's auc: 0.943674
[315]	valid_0's auc: 0.973391	valid_1's auc: 0.943682
[316]	valid_0's auc: 0.973464	valid_1's auc: 0.943688
[317]	valid_0's auc: 0.973519	valid_1's auc: 0.943659
[318]	valid_0's auc: 0.97359	valid_1's auc: 0.943683
[319]	valid_0's auc: 0.973646	valid_1's auc: 0.943705
[320]	valid_0's auc: 0.973696	valid_1's auc: 0.943715
[321]	valid_0's auc: 0.973766	valid_1's auc: 0.943705
[322]	valid_0's auc: 0.97384	valid_1's auc: 0.943694
[323]	valid_0's auc: 0.97391	valid_1's auc: 0.94374
[324]	valid_0's auc: 0.973971	valid_

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:547: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


[1]	valid_0's auc: 0.908536	valid_1's auc: 0.878292
Training until validation scores don't improve for 100 rounds.
[2]	valid_0's auc: 0.924511	valid_1's auc: 0.898346
[3]	valid_0's auc: 0.933533	valid_1's auc: 0.914815
[4]	valid_0's auc: 0.937219	valid_1's auc: 0.919247
[5]	valid_0's auc: 0.937564	valid_1's auc: 0.919852
[6]	valid_0's auc: 0.940066	valid_1's auc: 0.923546
[7]	valid_0's auc: 0.940692	valid_1's auc: 0.925027
[8]	valid_0's auc: 0.941326	valid_1's auc: 0.925915
[9]	valid_0's auc: 0.941521	valid_1's auc: 0.926045
[10]	valid_0's auc: 0.941408	valid_1's auc: 0.925734
[11]	valid_0's auc: 0.942434	valid_1's auc: 0.926765
[12]	valid_0's auc: 0.942536	valid_1's auc: 0.926716
[13]	valid_0's auc: 0.942672	valid_1's auc: 0.92685
[14]	valid_0's auc: 0.942926	valid_1's auc: 0.927291
[15]	valid_0's auc: 0.943292	valid_1's auc: 0.927301
[16]	valid_0's auc: 0.943185	valid_1's auc: 0.926696
[17]	valid_0's auc: 0.94357	valid_1's auc: 0.927495
[18]	valid_0's auc: 0.943672	valid_1's auc: 0.9

[306]	valid_0's auc: 0.974124	valid_1's auc: 0.935538
[307]	valid_0's auc: 0.974217	valid_1's auc: 0.935546
[308]	valid_0's auc: 0.974282	valid_1's auc: 0.935511
[309]	valid_0's auc: 0.974331	valid_1's auc: 0.935476
[310]	valid_0's auc: 0.974386	valid_1's auc: 0.935482
[311]	valid_0's auc: 0.974446	valid_1's auc: 0.93549
[312]	valid_0's auc: 0.974522	valid_1's auc: 0.935479
[313]	valid_0's auc: 0.974562	valid_1's auc: 0.935476
[314]	valid_0's auc: 0.974625	valid_1's auc: 0.935513
[315]	valid_0's auc: 0.974674	valid_1's auc: 0.935511
[316]	valid_0's auc: 0.974739	valid_1's auc: 0.935533
[317]	valid_0's auc: 0.974814	valid_1's auc: 0.935562
[318]	valid_0's auc: 0.974877	valid_1's auc: 0.935593
[319]	valid_0's auc: 0.974939	valid_1's auc: 0.935596
[320]	valid_0's auc: 0.974995	valid_1's auc: 0.935632
[321]	valid_0's auc: 0.97506	valid_1's auc: 0.935604
[322]	valid_0's auc: 0.97513	valid_1's auc: 0.935598
[323]	valid_0's auc: 0.97519	valid_1's auc: 0.93559
[324]	valid_0's auc: 0.975259	val

d:\anaconda3\lib\site-packages\ipykernel_launcher.py:547: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
d:\anaconda3\lib\site-packages\ipykernel_launcher.py:549: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
